## Examples of pySpark ML

In [91]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()
spark

In [93]:
# read dataset
training = spark.read.csv("/content/test6.csv", header=True, inferSchema=True)

training.show()
training.printSchema()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [94]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [96]:
# [Age, Experience]-----> new feature -----> independent feature

In [98]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=["age", "Experience"],
                                   outputCol="Independent Features")

output = featureassembler.transform(training)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [99]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [100]:
finalized_data = output.select("Independent Features", "Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [107]:
from pyspark.ml.regression import LinearRegression

# train test split
train_data,test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

In [108]:
# Coefficients

regressor.coefficients

DenseVector([-176.3341, 1320.1856])

In [109]:
# intercepts

regressor.intercept

19812.064965197555

In [110]:
# prediction
pred_results=regressor.evaluate(test_data)
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|17429.234338747123|
|         [31.0,10.0]| 30000|27547.563805104444|
+--------------------+------+------------------+



In [111]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(2440.8352668213392, 5957811.381290981)